In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

class GenderDetection:
    def __init__(self):
        # Load YOLOv8 model for person detection
        print("Loading YOLO model...")
        self.yolo_model = YOLO('yolov8n.pt')
       
        # Initialize Haar Cascade for face detection (built into OpenCV)
        print("Loading face detector...")
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
       
        # Gender classes
        self.gender_classes = ['Male', 'Female']
       
        # Age ranges
        self.age_ranges = ['Child (0-12)', 'Teen (13-19)', 'Young Adult (20-30)',
                          'Adult (31-50)', 'Middle-aged (51-65)', 'Senior (65+)']
       
        # Colors for visualization
        self.colors = {'Male': (255, 0, 0), 'Female': (255, 20, 147)}
        self.age_colors = {
            'Child (0-12)': (0, 255, 255),      # Cyan
            'Teen (13-19)': (255, 255, 0),       # Yellow
            'Young Adult (20-30)': (0, 255, 0),  # Green
            'Adult (31-50)': (255, 165, 0),      # Orange
            'Middle-aged (51-65)': (255, 0, 255), # Magenta
            'Senior (65+)': (128, 0, 128)        # Purple
        }
       
        print("Models loaded successfully!")
   
    def detect_people_yolo(self, frame):
        """Detect people using YOLO"""
        results = self.yolo_model(frame, classes=[0], verbose=False)  # class 0 is person
        people_boxes = []
       
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    conf = box.conf[0].cpu().numpy()
                   
                    if conf > 0.5:  # Confidence threshold
                        people_boxes.append([x1, y1, x2, y2, conf])
       
        return people_boxes
   
    def detect_faces_haar(self, frame):
        """Detect faces using Haar Cascade"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(50, 50))
       
        face_boxes = []
        for (x, y, w, h) in faces:
            face_boxes.append([x, y, x+w, y+h, 0.8])  # Adding dummy confidence
       
        return face_boxes
   
    def analyze_facial_features(self, face_crop):
        """Analyze facial features for gender and age classification"""
        if face_crop.size == 0:
            return 'Unknown', 0.5, 'Unknown', 0.5
       
        # Convert to grayscale
        gray = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
       
        if h < 50 or w < 50:
            return 'Unknown', 0.5, 'Unknown', 0.5
       
        # Feature extraction
        features = self.extract_features(gray, face_crop)
       
        # Classification based on features
        gender, gender_conf = self.classify_gender_by_features(features)
        age_range, age_conf = self.classify_age_by_features(features)
       
        return gender, gender_conf, age_range, age_conf
   
    def extract_features(self, gray_face, color_face):
        """Extract comprehensive features from face"""
        features = {}
       
        # Face dimensions
        h, w = gray_face.shape
        features['aspect_ratio'] = w / h if h > 0 else 1
        features['face_area'] = h * w
       
        # Divide face into regions
        upper_half = gray_face[:h//2, :]
        lower_half = gray_face[h//2:, :]
       
        # Eye area (upper third)
        eye_area = gray_face[:h//3, :]
       
        # Jaw area (lower third)
        jaw_area = gray_face[2*h//3:, :]
       
        # Forehead area (top quarter)
        forehead_area = gray_face[:h//4, :]
       
        # Calculate intensity variations (texture)
        features['upper_variance'] = np.var(upper_half)
        features['lower_variance'] = np.var(lower_half)
        features['jaw_variance'] = np.var(jaw_area) if jaw_area.size > 0 else 0
        features['eye_variance'] = np.var(eye_area) if eye_area.size > 0 else 0
        features['forehead_variance'] = np.var(forehead_area) if forehead_area.size > 0 else 0
       
        # Edge detection for feature sharpness
        edges = cv2.Canny(gray_face, 50, 150)
        features['edge_density'] = np.sum(edges) / (h * w)
       
        # Regional edge density
        upper_edges = edges[:h//2, :]
        lower_edges = edges[h//2:, :]
        features['upper_edge_density'] = np.sum(upper_edges) / upper_edges.size
        features['lower_edge_density'] = np.sum(lower_edges) / lower_edges.size
       
        # Histogram features
        hist = cv2.calcHist([gray_face], [0], None, [256], [0, 256])
        features['hist_peak'] = np.argmax(hist)
        features['hist_spread'] = np.std(hist)
        features['brightness_mean'] = np.mean(gray_face)
       
        # Wrinkle detection (high frequency content)
        features['wrinkle_score'] = self.detect_wrinkles(gray_face)
       
        # Skin texture analysis
        features['skin_smoothness'] = self.analyze_skin_texture(color_face)
       
        # Face symmetry
        features['symmetry_score'] = self.calculate_symmetry(gray_face)
       
        # Regional brightness variations
        features['brightness_variation'] = np.std([
            np.mean(eye_area),
            np.mean(jaw_area),
            np.mean(forehead_area)
        ])
       
        return features
   
    def detect_wrinkles(self, gray_face):
        """Detect wrinkles and fine lines for age estimation"""
        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray_face, (3, 3), 0)
       
        # Use Laplacian to detect fine details (wrinkles)
        laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
       
        # Calculate wrinkle score
        wrinkle_score = np.var(laplacian)
       
        return wrinkle_score
   
    def analyze_skin_texture(self, color_face):
        """Analyze skin texture for age estimation"""
        # Convert to LAB color space for better skin analysis
        lab = cv2.cvtColor(color_face, cv2.COLOR_BGR2LAB)
        l_channel = lab[:, :, 0]  # Lightness channel
       
        # Calculate local binary pattern-like texture measure
        kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])
        texture_response = cv2.filter2D(l_channel, -1, kernel)
       
        # Smoothness score (inverse of texture variation)
        smoothness = 1.0 / (1.0 + np.var(texture_response))
       
        return smoothness
   
    def calculate_symmetry(self, gray_face):
        """Calculate face symmetry"""
        h, w = gray_face.shape
       
        # Split face vertically
        left_half = gray_face[:, :w//2]
        right_half = gray_face[:, w//2:]
       
        # Flip right half
        right_flipped = cv2.flip(right_half, 1)
       
        # Resize to match if needed
        min_width = min(left_half.shape[1], right_flipped.shape[1])
        left_resized = left_half[:, :min_width]
        right_resized = right_flipped[:, :min_width]
       
        # Calculate similarity (symmetry)
        if left_resized.shape == right_resized.shape:
            difference = np.abs(left_resized.astype(float) - right_resized.astype(float))
            symmetry_score = 1.0 / (1.0 + np.mean(difference))
        else:
            symmetry_score = 0.5
       
        return symmetry_score
    def classify_gender_by_features(self, features):
        """Gender classification based on facial features"""
        male_score = 0
        female_score = 0
       
        # Feature 1: Aspect ratio (males tend to have wider faces)
        if features['aspect_ratio'] > 0.85:
            male_score += 0.3
        else:
            female_score += 0.3
       
        # Feature 2: Jaw variance (males tend to have more defined jaws)
        if features['jaw_variance'] > features['upper_variance'] * 1.2:
            male_score += 0.25
        else:
            female_score += 0.25
       
        # Feature 3: Edge density (facial hair, sharper features)
        if features['edge_density'] > 0.15:
            male_score += 0.2
        else:
            female_score += 0.2
       
        # Feature 4: Overall texture variance
        total_variance = features['upper_variance'] + features['lower_variance']
        if total_variance > 800:
            male_score += 0.15
        else:
            female_score += 0.15
       
        # Feature 5: Histogram characteristics
        if features['hist_peak'] < 120:  # Darker features
            male_score += 0.1
        else:
            female_score += 0.1
       
        # Determine gender
        if male_score > female_score:
            confidence = min(0.95, 0.5 + male_score)
            return 'Male', confidence
        else:
            confidence = min(0.95, 0.5 + female_score)
            return 'Female', confidence
   
    def classify_age_by_features(self, features):
        """Age classification based on facial features"""
        age_scores = {
            'Child (0-12)': 0,
            'Teen (13-19)': 0,
            'Young Adult (20-30)': 0,
            'Adult (31-50)': 0,
            'Middle-aged (51-65)': 0,
            'Senior (65+)': 0
        }
       
        # Feature 1: Face size (children have smaller faces)
        if features['face_area'] < 5000:
            age_scores['Child (0-12)'] += 0.4
        elif features['face_area'] < 8000:
            age_scores['Teen (13-19)'] += 0.3
            age_scores['Young Adult (20-30)'] += 0.2
        else:
            age_scores['Adult (31-50)'] += 0.2
            age_scores['Young Adult (20-30)'] += 0.2
       
        # Feature 2: Wrinkle score (increases with age)
        wrinkle_threshold = [100, 300, 600, 1000, 1500]
        if features['wrinkle_score'] < wrinkle_threshold[0]:
            age_scores['Child (0-12)'] += 0.3
            age_scores['Teen (13-19)'] += 0.2
        elif features['wrinkle_score'] < wrinkle_threshold[1]:
            age_scores['Teen (13-19)'] += 0.3
            age_scores['Young Adult (20-30)'] += 0.2
        elif features['wrinkle_score'] < wrinkle_threshold[2]:
            age_scores['Young Adult (20-30)'] += 0.3
            age_scores['Adult (31-50)'] += 0.2
        elif features['wrinkle_score'] < wrinkle_threshold[3]:
            age_scores['Adult (31-50)'] += 0.3
            age_scores['Middle-aged (51-65)'] += 0.2
        elif features['wrinkle_score'] < wrinkle_threshold[4]:
            age_scores['Middle-aged (51-65)'] += 0.3
            age_scores['Senior (65+)'] += 0.2
        else:
            age_scores['Senior (65+)'] += 0.4
       
        # Feature 3: Skin smoothness (decreases with age)
        if features['skin_smoothness'] > 0.7:
            age_scores['Child (0-12)'] += 0.2
            age_scores['Teen (13-19)'] += 0.2
        elif features['skin_smoothness'] > 0.5:
            age_scores['Teen (13-19)'] += 0.2
            age_scores['Young Adult (20-30)'] += 0.2
        elif features['skin_smoothness'] > 0.3:
            age_scores['Young Adult (20-30)'] += 0.2
            age_scores['Adult (31-50)'] += 0.2
        else:
            age_scores['Adult (31-50)'] += 0.15
            age_scores['Middle-aged (51-65)'] += 0.2
            age_scores['Senior (65+)'] += 0.2
       
        # Feature 4: Brightness variation (increases with age due to uneven skin)
        if features['brightness_variation'] < 10:
            age_scores['Child (0-12)'] += 0.1
            age_scores['Teen (13-19)'] += 0.1
        elif features['brightness_variation'] < 20:
            age_scores['Teen (13-19)'] += 0.1
            age_scores['Young Adult (20-30)'] += 0.1
        else:
            age_scores['Adult (31-50)'] += 0.1
            age_scores['Middle-aged (51-65)'] += 0.1
            age_scores['Senior (65+)'] += 0.1
       
        # Find the age range with highest score
        best_age = max(age_scores, key=age_scores.get)
        confidence = min(0.95, 0.4 + age_scores[best_age])
       
        return best_age, confidence
   
    def process_frame(self, frame):
        """Process single frame for gender detection"""
        # Detect faces using Haar Cascade
        faces = self.detect_faces_haar(frame)
       
        results = []
       
        for face_box in faces:
            x1, y1, x2, y2, conf = face_box
           
            # Add some padding
            padding = 10
            y1 = max(0, y1 - padding)
            y2 = min(frame.shape[0], y2 + padding)
            x1 = max(0, x1 - padding)
            x2 = min(frame.shape[1], x2 + padding)
           
            # Extract face region
            face_crop = frame[y1:y2, x1:x2]
           
            if face_crop.size > 0:
                try:
                    # Classify gender and age
                    gender, gender_conf, age_range, age_conf = self.analyze_facial_features(face_crop)
                   
                    if gender != 'Unknown':
                        results.append({
                            'box': [x1, y1, x2, y2],
                            'gender': gender,
                            'gender_confidence': float(gender_conf),
                            'age_range': age_range,
                            'age_confidence': float(age_conf),
                            'face_conf': float(conf)
                        })
                except Exception as e:
                    print(f"Error in gender classification: {e}")
                    continue
       
        return results
   
    def process_frame_with_people(self, frame):
        """Alternative method: detect people first, then faces"""
        people = self.detect_people_yolo(frame)
        results = []
       
        for person_box in people:
            x1, y1, x2, y2, person_conf = person_box
           
            # Extract person region
            person_crop = frame[y1:y2, x1:x2]
           
            # Detect faces in person region
            faces_in_person = self.detect_faces_haar(person_crop)
           
            for face in faces_in_person:
                fx1, fy1, fx2, fy2, _ = face
               
                # Adjust coordinates to full frame
                abs_fx1 = x1 + fx1
                abs_fy1 = y1 + fy1
                abs_fx2 = x1 + fx2
                abs_fy2 = y1 + fy2
               
                # Extract face
                face_crop = frame[abs_fy1:abs_fy2, abs_fx1:abs_fx2]
               
                if face_crop.size > 0:
                    try:
                        gender, gender_conf, age_range, age_conf = self.analyze_facial_features(face_crop)
                       
                        if gender != 'Unknown':
                            results.append({
                                'box': [abs_fx1, abs_fy1, abs_fx2, abs_fy2],
                                'person_box': [x1, y1, x2, y2],
                                'gender': gender,
                                'gender_confidence': float(gender_conf),
                                'age_range': age_range,
                                'age_confidence': float(age_conf),
                                'person_conf': float(person_conf)
                            })
                    except:
                        continue
       
        return results
   
    def draw_results(self, frame, results):
        """Draw bounding boxes and gender/age labels"""
        for result in results:
            x1, y1, x2, y2 = result['box']
            gender = result['gender']
            gender_conf = result['gender_confidence']
            age_range = result['age_range']
            age_conf = result['age_confidence']
           
            # Draw face bounding box (gender color)
            gender_color = self.colors.get(gender, (0, 255, 255))
            cv2.rectangle(frame, (x1, y1), (x2, y2), gender_color, 2)
           
            # Draw age range indicator (small rectangle)
            age_color = self.age_colors.get(age_range, (128, 128, 128))
            cv2.rectangle(frame, (x2-20, y1), (x2, y1+20), age_color, -1)
           
            # Draw person box if available
            if 'person_box' in result:
                px1, py1, px2, py2 = result['person_box']
                cv2.rectangle(frame, (px1, py1), (px2, py2), (128, 128, 128), 1)
           
            # Create labels
            gender_label = f"{gender}: {gender_conf:.2f}"
            age_label = f"{age_range}: {age_conf:.2f}"
           
            # Calculate label dimensions
            gender_size = cv2.getTextSize(gender_label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
            age_size = cv2.getTextSize(age_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
           
            # Draw gender label background
            cv2.rectangle(frame, (x1, y1-55), (x1+max(gender_size[0], age_size[0])+10, y1),
                         gender_color, -1)
           
            # Draw gender label text
            cv2.putText(frame, gender_label, (x1+5, y1-35),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
           
            # Draw age label text
            cv2.putText(frame, age_label, (x1+5, y1-15),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
       
        return frame

def main():
    try:
        # Initialize gender detection
        print("Initializing Gender Detection System...")
        detector = GenderDetection()
       
        # Open camera
        cap = cv2.VideoCapture(0)
       
        if not cap.isOpened():
            print("Error: Could not open camera. Trying with video file...")
            # You can replace this with a video file path
            cap = cv2.VideoCapture('sample_video.mp4')
           
        if not cap.isOpened():
            print("Error: Could not open camera or video file")
            return
       
        print("System ready! Press 'q' to quit, 'p' to switch modes")
        use_people_detection = False
       
        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video or camera error")
                break
           
            # Process frame
            if use_people_detection:
                results = detector.process_frame_with_people(frame)
                mode_text = "Mode: YOLO + Face Detection"
            else:
                results = detector.process_frame(frame)
                mode_text = "Mode: Direct Face Detection"
           
            # Draw results
            output_frame = detector.draw_results(frame, results)
           
            # Add info text
            info_text = f"Faces: {len(results)} | Gender & Age Detection"
            cv2.putText(output_frame, info_text,
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
           
            cv2.putText(output_frame, mode_text,
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
           
            # Age range legend
            legend_y = 90
            cv2.putText(output_frame, "Age Colors:", (10, legend_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
           
            for i, (age_range, color) in enumerate(detector.age_colors.items()):
                y_pos = legend_y + 20 + (i * 15)
                cv2.rectangle(output_frame, (10, y_pos-10), (25, y_pos), color, -1)
                cv2.putText(output_frame, age_range.split('(')[0], (30, y_pos-2),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
           
            cv2.putText(output_frame, "Press 'p' to switch modes, 'q' to quit",
                       (10, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
           
            # Display frame
            cv2.imshow('Gender Detection with YOLO', output_frame)
           
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('p'):
                use_people_detection = not use_people_detection
                print(f"Switched to {'YOLO + Face' if use_people_detection else 'Direct Face'} detection mode")
       
        cap.release()
        cv2.destroyAllWindows()
       
    except Exception as e:
        print(f"Error in main: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Initializing Gender Detection System...
Loading YOLO model...
Loading face detector...
Models loaded successfully!
System ready! Press 'q' to quit, 'p' to switch modes
